# Softmax Classifier using TensorFlow

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000017455C11A20>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001745DD51518>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001745DD51588>)

In [4]:
import numpy as np


In [5]:
print("number of data points : ", mnist.train.images.shape[0],"number of pixels in each image :",mnist.train.images.shape[1])

number of data points :  55000 number of pixels in each image : 784


In [6]:
print("number of data points : ", mnist.test.labels.shape[0]," length of the one hot encoded label vector :",mnist.test.labels.shape[1])

number of data points :  10000  length of the one hot encoded label vector : 10


In [7]:
import tensorflow as tf

In [8]:
# Get a list of devices like GPUs and CPUs available to TF

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17451210936124569635
]


### Placeholders and Variables

In [9]:
# We want to be able to input any number of MNIST images, 
# each flattened into a 784-dimensional vector. 

# We represent this as a 2-D tensor of floating-point numbers, 
# with a shape [None, 784]. 

# (Here None means that a dimension can be of any length.)
x = tf.placeholder(tf.float32, [None, 784])

# We also need the weights and biases for our model. 

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
# First, we multiply x by W with the expression tf.matmul(x, W). 
# We then add b, and finally apply tf.nn.softmax.

y = tf.nn.softmax(tf.matmul(x, W) + b)

In [12]:
y

<tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float32>

In [16]:
# y_ is true label of the images, and similar to x
y_ = tf.placeholder(tf.float32, [None, 10])


# Defining the loss function: multi class log-loss/cross-entropy
# First, tf.log computes the logarithm of each element of y. 
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [17]:
# We ask TensorFlow to minimize cross_entropy 
# using the gradient descent algorithm with a learning rate of 0.05.
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)


In [18]:
# We can now launch the model in an InteractiveSession
sess = tf.InteractiveSession()

# We first have to create an operation to initialize the 
# variables we created:
tf.global_variables_initializer().run()

In [19]:
# Each step of the loop, we get a "mini-batch" of one hundred random data 
# points from our training set. 

# We run train_step feeding in the batches data to replace 
# the placeholders
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [20]:
# tf.argmax(input, axis=None, name=None, dimension=None)
# Returns the index with the largest value across axis of a tensor.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9014
